# Zadania
1. Utwórz dwie nowe kolumny w ramce "Countries", w których powierzchnia i gęstość zaludnienia wyrażone sa odpowiednio w milach kwadratowych i osobach na milę kwadratową. 
  - Przelicznik: 1 mila kwadratowa = 0.38610 km kwadratowego (powierzchnię należy pomnożyć, a gęstość zaludnienia podzielić przez ten współczynnik)
  - Typami nowych kolumn mają być odpowiednio integer (powierzchnia) i float (gęstość). Zaokrąglanie do integer: "int(x)"
  - Nazwy nowych kolumn proszę ustawić odpowiednio na '<tt>Area (sq mi)</tt>' oraz '<tt>Pop Density (per sq mi)</tt>'
2. Na wykresie scatterplot narysuj zależność liczby lotnisk od powierzchni kraju. 
  - Dane potrzebne do wykresu są w dwóch ramkach (Countries i Airports), konieczne więc będzie ich złączenie (join).
  - Wskazówka: w pierwszym etapie stwórz ramkę agregującą liczbę lotnisk w zależności od kraju. W drugim etapie złącz (join) tę ramkę z ramką countries. Kolumną wspólną (warunkiem złączenia) będzie Country.
  - Nie zawsze nazwy tych samych krajów będą identyczne w obu ramkach, co może stanowić problem (złączenie nie powstanie). Zastanów się jak przy pomocy mechanizmu złączeń wykryć, które nazwy się różnią. Stwórz odpowiednią ramkę, która zawiera takie nazwy. 
  - Następnie zmień nazwy kilku największych państw w jednej z ramek, tak żeby ujednolicić nazwy. Można to zrobić funkcją replace:
<br><tt>cdf=cdf.replace(['Korea, South', 'Korea, North'], ['South Korea', 'North Korea'], 'Country')</tt>
3. Do ramki "Countries" dodaj nową kolumnę "Continent" (według klasyfikacji Africa, Asia, Europe, North America, South America, Antarctica, Australia/Oceania). 
  - Skorzystaj z kolumny "Region" i pomocniczej funkcji (UDF) mapującej region na kontynent.
4. Oblicz, ile lotnisk jest na poszczególnych kontynentach. To zadanie również wymaga złączenia ramek "Countries" i "Airports". Wynik przedstaw na wykresie słupkowym.


In [402]:
from pyspark.sql import SparkSession

In [403]:
import pandas as pd

In [404]:
import numpy as np

In [405]:
spark = SparkSession.builder.getOrCreate()

In [406]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [407]:
# Wbudowane funkcje, które mogą być przydatne:
# - udf: służy do tworzenia funkcji użytkownika
# - trim: usuwa białe spacje z początku i końca stringu
# - isnull: testuje, czy wartość jest pusta
from pyspark.sql.functions import col, udf, trim, isnull
from pyspark.sql.types import FloatType, IntegerType

In [408]:
def to_float (s) :
   return float(s.replace(',','.'))
float_udf = udf(to_float , FloatType())

In [409]:
countries = spark.read.csv("countries of the world.csv", inferSchema=True, header=True)

# Usuwa kropki z wszystkich nazw kolumn (powodują wiele błędów, prawdopodobnie bug w Sparku)
# (Nb. wszystkie inne metody zmiany tych nazw (np. użycie columnRenamed) zawiodły)
new_columns = [s.replace('.','') for s in countries.columns]
countries = countries.toDF(*new_columns) # tutaj '*' to operator "splat" -- robi z tablicy listę argumentów

# Tak można ustawić wyświetlanie większej liczby wierszy w Jupyterze
pd.set_option('display.max_rows', 20)
display(countries.toPandas())

,Country,Region,Population,Area (sq mi),Pop Density (per sq mi),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",None,None,None
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,West Bank,NEAR EAST,2460492,5860,"419,9","0,00","2,98","19,62",800.0,None,"145,2","16,9","18,97","64,13",3,"31,67","3,92","0,09","0,28","0,63"
223,Western Sahara,NORTHERN AFRICA,273008,266000,"1,0","0,42",None,None,NaN,None,None,"0,02",0,"99,98",1,None,None,None,None,"0,4"
224,Yemen,NEAR EAST,21456188,527970,"40,6","0,36",0,"61,5",800.0,"50,2","37,2","2,78","0,24","96,98",1,"42,89","8,3","0,135","0,472","0,393"
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,"15,3","0,00",0,"88,29",800.0,"80,6","8,2","7,08","0,03","92,9",2,41,"19,93","0,22","0,29","0,489"


In [410]:
# Utworzenie nowej ramki 'cdf':
# - wartosci puste wypelnione "-1", zeby nie usuwac wierszy
# - trim stringów (w nazwach występują niepożądane spacje na końcu)
# - konwersja wszystkich kolumn zmiennoprzecinkowych na float
# - zmiana 'sq mi' na 'sq km', bo w zbiorze wartości są faktycznie w km^2, a nie milach^2
cdf = countries.na.fill("-1").select( 
                 trim(col('Country')).alias('Country'),\
                 trim(col('Region')).alias('Region'),'Population',\
                 col('`Area (sq mi)`').alias('Area (sq km)'),\
                 float_udf('`Pop Density (per sq mi)`').alias('Pop Density (per sq km)'),\
                 float_udf('Coastline (coast/area ratio)').alias('Coastline (coast/area ratio)'),\
                 float_udf('Net migration').alias('Net migration'),\
                 float_udf('Infant mortality (per 1000 births)').alias('Infant mortality (per 1000 births)'),\
                 'GDP ($ per capita)',\
                 float_udf('Literacy (%)').alias('Literacy (%)'),\
                 float_udf('Phones (per 1000)').alias('Phones (per 1000)'),\
                 float_udf('Arable (%)').alias('Arable (%)'),\
                 float_udf('Crops (%)').alias('Crops (%)'),\
                 float_udf('Other (%)').alias('Other (%)'),\
                 float_udf('Birthrate').alias('Birthrate'),\
                 float_udf('Deathrate').alias('Deathrate'),\
                 float_udf('Agriculture').alias('Agriculture'),\
                 float_udf('Industry').alias('Industry'),\
                 float_udf('Service').alias('Service'))

# Zadanie 1: nowe kolumny

In [411]:
cdf.toPandas()

,Country,Region,Population,Area (sq km),Pop Density (per sq km),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.000000,0.000000,23.059999,163.070007,700.0,36.000000,3.200000,12.13,0.220000,87.650002,46.599998,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.599998,1.260000,-4.930000,21.520000,4500.0,86.500000,71.199997,21.09,4.420000,74.489998,15.110000,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.800000,0.040000,-0.390000,31.000000,6000.0,70.000000,78.099998,3.22,0.250000,96.529999,17.139999,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.399994,58.290001,-20.709999,9.270000,8000.0,97.000000,259.500000,10.00,15.000000,75.000000,22.459999,3.27,-1.000,-1.000,-1.000
4,Andorra,WESTERN EUROPE,71201,468,152.100006,0.000000,6.600000,4.050000,19000.0,100.000000,497.200012,2.22,0.000000,97.779999,8.710000,6.25,-1.000,-1.000,-1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,West Bank,NEAR EAST,2460492,5860,419.899994,0.000000,2.980000,19.620001,800.0,-1.000000,145.199997,16.90,18.969999,64.129997,31.670000,3.92,0.090,0.280,0.630
223,Western Sahara,NORTHERN AFRICA,273008,266000,1.000000,0.420000,-1.000000,-1.000000,NaN,-1.000000,-1.000000,0.02,0.000000,99.980003,-1.000000,-1.00,-1.000,-1.000,0.400
224,Yemen,NEAR EAST,21456188,527970,40.599998,0.360000,0.000000,61.500000,800.0,50.200001,37.200001,2.78,0.240000,96.980003,42.889999,8.30,0.135,0.472,0.393
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,15.300000,0.000000,0.000000,88.290001,800.0,80.599998,8.200000,7.08,0.030000,92.900002,41.000000,19.93,0.220,0.290,0.489


In [412]:
cdf = cdf.withColumn('Area (sq mi)', col('Area (sq km)') * 0.38610)\
    .withColumn('Pop Density (per sq mi)', col('Pop Density (per sq km)') / 0.38610)

In [413]:
cdf = cdf.withColumn('Area (sq mi)', cdf['Area (sq mi)'].cast(IntegerType()))

In [414]:
cdf.toPandas()

,Country,Region,Population,Area (sq km),Pop Density (per sq km),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),...,Arable (%),Crops (%),Other (%),Birthrate,Deathrate,Agriculture,Industry,Service,Area (sq mi),Pop Density (per sq mi)
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.000000,0.000000,23.059999,163.070007,700.0,36.000000,...,12.13,0.220000,87.650002,46.599998,20.34,0.380,0.240,0.380,249999,124.320124
1,Albania,EASTERN EUROPE,3581655,28748,124.599998,1.260000,-4.930000,21.520000,4500.0,86.500000,...,21.09,4.420000,74.489998,15.110000,5.22,0.232,0.188,0.579,11099,322.714319
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.800000,0.040000,-0.390000,31.000000,6000.0,70.000000,...,3.22,0.250000,96.529999,17.139999,4.61,0.101,0.600,0.298,919589,35.742036
3,American Samoa,OCEANIA,57794,199,290.399994,58.290001,-20.709999,9.270000,8000.0,97.000000,...,10.00,15.000000,75.000000,22.459999,3.27,-1.000,-1.000,-1.000,76,752.136736
4,Andorra,WESTERN EUROPE,71201,468,152.100006,0.000000,6.600000,4.050000,19000.0,100.000000,...,2.22,0.000000,97.779999,8.710000,6.25,-1.000,-1.000,-1.000,180,393.939410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,West Bank,NEAR EAST,2460492,5860,419.899994,0.000000,2.980000,19.620001,800.0,-1.000000,...,16.90,18.969999,64.129997,31.670000,3.92,0.090,0.280,0.630,2262,1087.542072
223,Western Sahara,NORTHERN AFRICA,273008,266000,1.000000,0.420000,-1.000000,-1.000000,NaN,-1.000000,...,0.02,0.000000,99.980003,-1.000000,-1.00,-1.000,-1.000,0.400,102702,2.590003
224,Yemen,NEAR EAST,21456188,527970,40.599998,0.360000,0.000000,61.500000,800.0,50.200001,...,2.78,0.240000,96.980003,42.889999,8.30,0.135,0.472,0.393,203849,105.154101
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,15.300000,0.000000,0.000000,88.290001,800.0,80.599998,...,7.08,0.030000,92.900002,41.000000,19.93,0.220,0.290,0.489,290584,39.627040


# Join examples

In [415]:
valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
TableA = spark.createDataFrame(valuesA,['name','id'])
 
valuesB = [('Rutabaga',1),('Pirate',2),('Ninja',3),('Darth Vader',4)]
TableB = spark.createDataFrame(valuesB,['name','id'])

In [416]:
a = TableA.alias('a')
b = TableB.alias('b')

In [417]:
a.join(b, a.name==b.name).show() # how='inner'

+------+---+------+---+
|  name| id|  name| id|
+------+---+------+---+
| Ninja|  3| Ninja|  3|
|Pirate|  1|Pirate|  2|
+------+---+------+---+



In [418]:
a.join(b, a.name==b.name, how='left').show()

+---------+---+------+----+
|     name| id|  name|  id|
+---------+---+------+----+
|Spaghetti|  4|  null|null|
|    Ninja|  3| Ninja|   3|
|   Pirate|  1|Pirate|   2|
|   Monkey|  2|  null|null|
+---------+---+------+----+



In [419]:
a.join(b, a.name==b.name, how='right').show()

+------+----+-----------+---+
|  name|  id|       name| id|
+------+----+-----------+---+
|  null|null|   Rutabaga|  1|
| Ninja|   3|      Ninja|  3|
|Pirate|   1|     Pirate|  2|
|  null|null|Darth Vader|  4|
+------+----+-----------+---+



In [420]:
a.join(b, a.name==b.name, how='full_outer').show()

+---------+----+-----------+----+
|     name|  id|       name|  id|
+---------+----+-----------+----+
|     null|null|   Rutabaga|   1|
|Spaghetti|   4|       null|null|
|    Ninja|   3|      Ninja|   3|
|   Pirate|   1|     Pirate|   2|
|   Monkey|   2|       null|null|
|     null|null|Darth Vader|   4|
+---------+----+-----------+----+



# Zadanie 2: zależność liczby lotnisk od powierzchni kraju 

In [421]:
airports = spark.read.csv("airports2.csv", inferSchema=True, header=False)

In [422]:
airports = airports.toDF('Airport ID', 'Name', 'City', 'Country', 
'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone',
'DST', 'Tz database time zone', 'Type', 'Source'
)

In [423]:
airports.toPandas()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7179,12053,Rugao Air Base,Rugao,China,RUG,ZSRG,32.257885,120.501656,0,\N,\N,\N,airport,OurAirports
7180,12054,Wuhu Air Base,Wuhu,China,WHU,ZSWU,31.390600,118.408997,0,\N,\N,\N,airport,OurAirports
7181,12055,Shanshan Airport,Shanshan,China,SXJ,ZWSS,42.911701,90.247498,0,\N,\N,\N,airport,OurAirports
7182,12056,Yingkou Lanqi Airport,Yingkou,China,YKH,ZYYK,40.542524,122.358600,0,\N,\N,\N,airport,OurAirports


In [424]:
# TODO
# zacznij od stworzenia ramki, która zawiera posortowaną liczbę lotnisk w poszczególnych krajach

In [425]:
import pyspark.sql.functions as func

In [426]:
sumAirports = airports \
    .groupBy("Country") \
    .agg(func.count("Name").alias("Airports sum")) \
    .select("Country", "Airports sum") \
    .sort("Airports sum") \
    .withColumn("Country", trim(col("Country")))

In [427]:
countries = countries.replace(['Korea, South', 'Korea, North', 'South Korea', 'North Korea'], 'Country')

In [428]:
countries = countries \
    .withColumn("Country", trim(col("Country")))

In [429]:
sumAirports.toPandas()

,Country,Airports sum
0,Anguilla,1
1,British Indian Ocean Territory,1
2,Jersey,1
3,Norfolk Island,1
4,Saint Helena,1
...,...,...
232,Russia,238
233,Germany,241
234,Australia,296
235,Canada,417


In [430]:
# TODO złączenie ramek i wykres

In [431]:
a = countries.alias('a')
b = sumAirports.alias('b')

In [439]:
joined = b.join(a, on=['Country'], how='inner')

In [440]:
joined.toPandas()

,Country,Airports sum,Region,Population,Area (sq mi),Pop Density (per sq mi),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),...,Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Chad,6,SUB-SAHARAN AFRICA,9944201,1284000,"7,7","0,00","-0,11","93,82",1200.0,...,"1,3","2,86","0,02","97,12",2,"45,73","16,38","0,335","0,259","0,406"
1,Paraguay,8,LATIN AMER. & CARIB,6506464,406750,"16,0","0,00","-0,08","25,63",4700.0,...,"49,2","7,6","0,23","92,17",2,"29,1","4,49","0,224","0,207","0,569"
2,Anguilla,1,LATIN AMER. & CARIB,13477,102,"132,1","59,80","10,76","21,03",8600.0,...,"460,0",0,0,100,2,"14,17","5,34","0,04","0,18","0,78"
3,Russia,238,C.W. OF IND. STATES,142893540,17075200,"8,4","0,22","1,02","15,39",8900.0,...,"280,6","7,33","0,11","92,56",None,"9,95","14,65","0,054","0,371","0,575"
4,Yemen,11,NEAR EAST,21456188,527970,"40,6","0,36",0,"61,5",800.0,...,"37,2","2,78","0,24","96,98",1,"42,89","8,3","0,135","0,472","0,393"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,United Kingdom,162,WESTERN EUROPE,60609153,244820,"247,6","5,08","2,19","5,16",27700.0,...,"543,5","23,46","0,21","76,33",3,"10,71","10,13","0,005","0,237","0,758"
201,Moldova,5,C.W. OF IND. STATES,4466706,33843,"132,0","0,00","-0,26","40,42",1800.0,...,"208,1","55,3","10,79","33,91",None,"15,7","12,64","0,213","0,233","0,555"
202,Vietnam,24,ASIA (EX. NEAR EAST),84402966,329560,"256,1","1,05","-0,45","25,95",2500.0,...,"187,7","19,97","5,95","74,08",2,"16,86","6,22","0,209","0,41","0,381"
203,Netherlands,26,WESTERN EUROPE,16491461,41526,"397,1","1,09","2,91","5,04",28600.0,...,"460,8","26,71","0,97","72,32",3,"10,9","8,68","0,021","0,244","0,736"


In [446]:
joined_anti = a.join(b, on=['Country'], how='leftanti')

In [447]:
joined_anti.toPandas()

,Country,Region,Population,Area (sq mi),Pop Density (per sq mi),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",None,None,None
1,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,"156,0","34,54","-6,15","19,46",11000,"89,0","549,9","18,18","4,55","77,27",2,"16,93","5,37","0,038","0,22","0,743"
2,"Bahamas, The",LATIN AMER. & CARIB,303770,13940,"21,8","25,41","-2,2","25,21",16700,"95,6","460,6","0,8","0,4","98,8",2,"17,57","9,05","0,03","0,07","0,9"
3,Bosnia & Herzegovina,EASTERN EUROPE,4498976,51129,"88,0","0,04","0,31","21,05",6100,None,"215,4","13,6","2,96","83,44",4,"8,77","8,27","0,142","0,308","0,55"
4,British Virgin Is.,LATIN AMER. & CARIB,23098,153,"151,0","52,29","10,01","18,05",16000,"97,8","506,5",20,"6,67","73,33",2,"14,89","4,42","0,018","0,062","0,92"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,St Pierre & Miquelon,NORTHERN AMERICA,7026,242,"29,0","49,59","-4,86","7,54",6900,"99,0","683,2","13,04",0,"86,96",None,"13,52","6,83",None,None,None
18,San Marino,WESTERN EUROPE,29251,61,"479,5","0,00","10,98","5,73",34600,"96,0","704,3","16,67",0,"83,33",None,"10,02","8,17",None,None,None
19,Sao Tome & Principe,SUB-SAHARAN AFRICA,193413,1001,"193,2","20,88","-2,72","43,11",1200,"79,3","36,2","6,25","48,96","44,79",2,"40,25","6,47","0,167","0,148","0,684"
20,Trinidad & Tobago,LATIN AMER. & CARIB,1065842,5128,"207,9","7,06","-10,83","24,31",9500,"98,6","303,5","14,62","9,16","76,22",2,"12,9","10,57","0,007","0,57","0,423"


In [438]:
sumAirports.toPandas().to_csv(r'sumAirports.csv', index = False)
joined.toPandas().to_csv(r'joined.csv', index = False)



In [434]:
# TODO wykrycie niepasujących nazw państwa i ujednolicenie wartości 

# Zadanie 3

In [435]:
# TODO

# Zadanie 4

In [436]:
# TODO